# ADALA Quickstart

In this notebook, we are going to run through some of the common tasks for creating data labeling agents with ADALA. In this example, we're going to create a data labeling agent for a text classification task - labeling our text samples as either "Subjective or "Objective" statements. 

This agent will be LLM-based, so we will use [OpenAI's API](https://platform.openai.com/). You will to generate an API key and set it as an environment variable as follows: 

```
export OPENAI_API_KEY=your_openai_api_key
```

Now, let's begin. 

## Dataset Creation
First, let's use a dataset of product reviews stored in pandas dataframe. This will help us manage our data as we add more attributes, like predictions and labels for subjectivity and objectivity over time. 

In [1]:
import pandas as pd

df = pd.DataFrame([
    ["The mic is great.", "Subjective"],
    ["Will order from them again!", "Subjective"],
    ["Not loud enough and doesn't turn on like it should.", "Objective"],
    ["The phone doesn't seem to accept anything except CBR mp3s", "Objective"],
    ["All three broke within two months of use.", "Objective"]
], columns=["text", "ground_truth"])

df

,text,ground_truth
0,The mic is great.,Subjective
1,Will order from them again!,Subjective
2,Not loud enough and doesn't turn on like it sh...,Objective
3,The phone doesn't seem to accept anything exce...,Objective
4,All three broke within two months of use.,Objective


We instantiate Dataset that uses this pandas dataframe as a data source. Dataset object takes care of input data schema and data streaming:

In [2]:
from adala.datasets import DataFrameDataset

dataset = DataFrameDataset(df=df)

## Create Agent

To create Agent, we need to to define 2 things:

**Skills** - Agent's abilities are defined as _Skills_. Each agent can possess many different skills. In our case, this agent only has one labeling skill, to produce a classification of Subjective or Objective for a given piece of text.  To define this skill, we will leverage an LLM, passing it instructions and the set of labeles we expect to receive back. 

**Environment** - that is where the Agent receives ground truth signal to improve its skill. Since we already created ground truth dataset, we can simply refer to the column from the dataframe. In the real world scenario, you may consider using a different environment where ground truth signal can be obtained asynchoronously by gathering real human feedback during agent's learning phase.

In [3]:
from adala.agents import Agent
from adala.environments import BasicEnvironment
from adala.skills import ClassificationSkill
from adala.runtimes import OpenAIRuntime
from rich import print


agent = Agent(
    # define the agent's labeling skill that should classify text onto 2 categories
    skills=ClassificationSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels=['Subjective', 'Objective'],
        input_data_field='text'
    ),
    
    # basic environment extracts ground truth signal from the input records
    environment=BasicEnvironment(
        ground_truth_dataset=dataset,
        ground_truth_column='ground_truth'
    ),
    
    runtimes = {
        # You can specify your OPENAI API KEY here via `OpenAIRuntime(..., api_key='your-api-key')`
        'openai': OpenAIRuntime(model='gpt-3.5-turbo-instruct'),
        'openai-gpt3': OpenAIRuntime(model='gpt-3.5-turbo'),
        'openai-gpt4': OpenAIRuntime(model='gpt-4'),
    },
    default_runtime='openai',
    
    # NOTE! If you don't have an access to gpt4 - replace it with "openai-gpt3"
    default_teacher_runtime='openai-gpt4'
)

print(agent)

Agent Instance

Environment: BasicEnvironment
Skills: subjectivity_detection
Runtimes: openai, openai-gpt3, openai-gpt4
Default Runtime: openai
Default Teacher Runtime: openai-gpt4

## Learning Agent

We will now let Agent learn from the ground truth. After every action, Agent returns its _Experience_, where it stores various observations like predicted data, errors, accuracy, etc.

In [4]:
learning_experience = agent.learn(learning_iterations=3, accuracy_threshold=0.95)

100%|█████████████████| 5/5 [00:00<00:00, 39.18it/s]


=> Iteration #0: Comparing to ground truth, analyzing and improving ...

Comparing predictions to ground truth data ...

 text                     ground_truth   subjectivity_detection   score                    ground_truth__x__sub…  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  The mic is great.        Subjective     Subjective               {'Subjective':           True                   
                                                                   -0.02697588099999997,                           
                                                                   'Objective':                                    
                                                                   -3.6262724}                                     
  Will order from them     Subjective     Subjective               {'Subjective':           True                   
  again!                                                           -0.11282212000000001,                           
                                                                   'Objective':                                    
                                                                   -2.2378219999999995}                            
  Not loud enough and      Objective      Subjective               {'Subjective':           False                  
  doesn't turn on like                                             -0.014163457000000034,                          
  it should.                                                       'Objective':                                    
                                                                   -4.2641635}                                     
  The phone doesn't seem   Objective      Objective                {'Subjective':           True                   
  to accept anything                                               -2.0720863,                                     
  except CBR mp3s                                                  'Objective':                                    
                                                                   -0.13458653999999995}                           
  All three broke within   Objective      Objective                {'Subjective':           True                   
  two months of use.                                               -2.1821797,                                     
                                                                   'Objective':                                    
                                                                   -0.11967964500000007}

Analyze evaluation experience ...

...

Analyze evaluation experience ...

Number of errors: 0

Accuracy = 100.00%

Accuracy threshold reached (1.0 >= 0.95)

Train is done!

Let's see the final instructions:

In [5]:
print(agent.skills)

Total Agent Skills: 1

subjectivity_detection
Identify if the provided product review is "Subjective" (expressing personal feelings, tastes, or opinions) or 
"Objective" (based on factual information). Consider a statement as subjective if it reflects personal judgment or 
preference, and as objective if it states verifiable facts or features.

Examples:

Input: Not loud enough and doesn't turn on like it should.
Output: Objective

Input: I personally think the sound quality is not up to the mark.
Output: Subjective

Input: The phone's battery lasts for 10 hours.
Output: Objective

Input: The mic is great.
Output: Subjective

Input: Will order from them again!
Output: Subjective

... and predictions created by the skill:

In [6]:
learning_experience.predictions

,text,ground_truth,subjectivity_detection,score
0,The mic is great.,Subjective,Subjective,"{'Subjective': -0.022607480000000055, 'Objecti..."
1,Will order from them again!,Subjective,Subjective,"{'Subjective': -0.05627503599999997, 'Objectiv..."
2,Not loud enough and doesn't turn on like it sh...,Objective,Objective,"{'Subjective': -2.897738, 'Objective': -0.0567..."
3,The phone doesn't seem to accept anything exce...,Objective,Objective,"{'Subjective': -3.8168292, 'Objective': -0.022..."
4,All three broke within two months of use.,Objective,Objective,"{'Subjective': -4.800799, 'Objective': -0.0082..."


## Applying learned skills to the real data

Now as we have our Agent with evolved "subjectivity detection" skill, we can apply it to the real dataset without ground truth data:

In [7]:
test_df = pd.DataFrame([
    "Doesn't hold charge.",
    "Excellent bluetooth headset",
    "I love this thing!",
    "VERY DISAPPOINTED."
], columns=['text'])
test_df

,text
0,Doesn't hold charge.
1,Excellent bluetooth headset
2,I love this thing!
3,VERY DISAPPOINTED.


In [8]:
result = agent.apply_skills(test_df)

100%|█████████████████| 4/4 [00:00<00:00, 32.32it/s]


In [9]:
result.predictions

,text,subjectivity_detection,score
0,Doesn't hold charge.,Objective,"{'Subjective': -4.9062243, 'Objective': -0.007..."
1,Excellent bluetooth headset,Objective,"{'Subjective': -1.450324, 'Objective': -0.2672..."
2,I love this thing!,Subjective,"{'Subjective': -0.0014673689999999905, 'Object..."
3,VERY DISAPPOINTED.,Subjective,"{'Subjective': -0.17851222999999997, 'Objectiv..."
